## settings

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver
!pip install chromedriver-autoinstaller
!python --version

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://ppa.launchpadcontent.net

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
import time
import os
import requests

## 정책 용어 크롤링

### 청년정보 용어사전

In [ ]:
import pandas as pd

# ChromeDriver 자동 설치
chromedriver_autoinstaller.install()

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_argument('--headless')  # GUI 없이 실행 (headless 모드)
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('lang=ko_KR')

# ChromeDriver 실행
driver = webdriver.Chrome(options=chrome_options)

# 페이지 접속
url = "https://youthdb.notion.site/4c37824bc8ca448cb89f0403e133d7a4"
driver.get(url)
time.sleep(2)  # 페이지 로드 대기

wait = WebDriverWait(driver, 10)

terms = []
definitions = []

# 반복을 통해 용어와 뜻을 가져오기
for i in range(2, 60):  # 최대 60개의 항목을 탐색한다고 가정
    try:
        # 용어 부분의 Xpath
        term_xpath = f'//*[@id="notion-app"]/div/div[1]/div/div[1]/main/div/div/div[2]/div/div[4]/div/div[2]/div[3]/div/div/div/div/div[3]/div[{i}]/div[1]/div/div[3]/div/div/a/span'
        term_element = wait.until(EC.presence_of_element_located((By.XPATH, term_xpath)))
        term_text = term_element.text

        # 뜻 부분의 Xpath
        definition_xpath = f'//*[@id="notion-app"]/div/div[1]/div/div[1]/main/div/div/div[2]/div/div[4]/div/div[2]/div[3]/div/div/div/div/div[3]/div[{i}]/div[1]/div/div[4]/div/div/div/span'
        definition_element = wait.until(EC.presence_of_element_located((By.XPATH, definition_xpath)))
        definition_text = definition_element.text

        # 리스트에 추가
        terms.append(term_text)
        definitions.append(definition_text)

    except Exception as e:
        print(f"{i}번째 항목에서 오류 발생: {e}")
        break  # 더 이상 항목이 없으면 중단

# 데이터프레임에 저장
df = pd.DataFrame({
    "용어": terms,
    "뜻": definitions
})

# 크롤링된 데이터 출력
print(df)

# CSV 파일로 저장
df.to_csv("청년정보_용어사전.csv", index=False, encoding='utf-8-sig')

# WebDriver 종료
driver.quit()

                      용어                                                  뜻
0               4대(사회)보험  국민에게 발생할 수 있는 질병,상해,실업,노령의 위험을 사전에 미리 대비하고 이후 ...
1                   가산금리  금융기관에서 돈을 빌릴 때 사람, 대출상품의 종류에 따라 각각 다른 금리를 적용하는...
2                감정평가(액)  동산(차량, 기기 등), 부동산(토지, 건물 등)과 같은 재산의 경제적 가치를 판단...
3                   건강보험  국가에 매월 일정 금액을 미리 내고, 나중에 병원에 가서 아픈 곳을 진단하거나 치료...
4                 공공임대주택  정부나 민간 기업에서 주택을 구매하고, 이를 저소득층, 대학생, 청년, 신혼부부, ...
5              공공지원 임대주택  "임대=빌리다" 임대주택은 뜻 그대로 자신이 가진 집이 아닌 빌린 집을 말해요.\n...
6                   공급면적  방, 거실(전용 면적)에 계단, 현관(공용 면적) 등 주거와 관련된 모든 공간을 합...
7                   공동주택  하나의 건물에 여러 가구가 개별 공간에서 생활할 수 있도록 지어진 집을 말해요.\n...
8                   공용면적  말 그대로 불특정 다수가 공동으로 사용하는 바닥 부분의 넓이와 크기를 말해요.\n공...
9                   과세표준  세금을 부과하는 기준이 되는 수량, 가격 품질 등을 뜻해요. 보통 소득 구간에 따라...
10                    금리  은행이나 금융기관에서 빌린 돈, 내가 맡긴 돈에 생겨나는 이자나 그 이자의 비율을 ...
11                  기준금리  말 그대로 금리를 결정하기 위해 기준이 되는 체계를 말해요. 이 기준 금리는 국가별...
12         기

### 주택청약 용어 (청약홈)

In [ ]:
# ChromeDriver 자동 설치
chromedriver_autoinstaller.install()

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_argument('--headless')  # GUI 없이 실행 (headless 모드)
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('lang=ko_KR')

# ChromeDriver 실행
driver = webdriver.Chrome(options=chrome_options)

# 페이지 접속
url = "https://www.applyhome.co.kr/ar/ard/selectSubscriptVocabularyView.do#voca1"
driver.get(url)
time.sleep(2)

# WevDriverWait
wait = WebDriverWait(driver, 10)

terms = []
definitions = []

for i in range(1, 40):
    try:
        # 용어 클릭
        term_xpath = f'//*[@id="voca1"]/table/tbody/tr[{i}]/td[1]/a/span'
        term_element = wait.until(EC.presence_of_element_located((By.XPATH, term_xpath)))
        term_text = term_element.text

        # JavaScript 사용
        driver.execute_script("arguments[0].click();", term_element)
        time.sleep(1)

        # 용어 id 추출
        dialog_xpath = '//div[contains(@class, "ui-dialog-content")]'
        dialog_element = wait.until(EC.presence_of_element_located((By.XPATH, dialog_xpath)))
        policy_id = dialog_element.get_attribute("id")

        # 용어 설명 텍스트 추출
        definition_xpath = f'//*[@id="{policy_id}"]/div'
        definition_element = wait.until(EC.presence_of_element_located((By.XPATH, definition_xpath)))
        definition_text = definition_element.text

        # 리스트에 추가
        terms.append(term_text)
        definitions.append(definition_text)

        # 설명 창 닫기
        close_button_xpath = '/html/body/div[4]/div[1]/button'
        close_button = wait.until(EC.element_to_be_clickable((By.XPATH, close_button_xpath)))
        close_button.click()
        time.sleep(1)  # 창 닫기 대기

    except Exception as e:
        print(f"{i}번째 항목에서 오류 발생: {e}")
        break

# 데이터프레임에 저장
df = pd.DataFrame({
    "용어": terms,
    "설명": definitions
})

# 크롤링된 데이터 출력
print(df)

# CSV 파일로 저장
df.to_csv('주택청약 용어.csv', index = False, encoding='utf-8-sig')

# WebDriver 종료
driver.quit()

                            용어  \
0                         국민주택   
1                         민영주택   
2                         공공주택   
3                     공공건설임대주택   
4                     도시형 생활주택   
5                         오피스텔   
6                  공공지원 민간임대주택   
7                  청약통장(입주자저축)   
8                     주택청약종합저축   
9                         청약저축   
10                        청약예금   
11                        청약부금   
12                    무주택세대구성원   
13                          세대   
14                        부양가족   
15                         가점제   
16                        청약순위   
17                    지역별 예치금액   
18  소형·저가주택 등 소유자를 무주택자로 보는 경우   
19           주택 처분 시 무주택기간 기준일   
20                      자산보유기준   
21                      과밀억제권역   
22                      청약과열지역   
23                조정대상지역(과열지역)   
24                      투기과열지구   
25                대규모 택지개발지구 등   
26           대규모 택지개발지구 등 우선공급   
27                  (해당)주택건설지역   
28          해당

### 생활법령 (주거, 복지, 근로)

In [ ]:
# ChromeDriver 자동 설치
chromedriver_autoinstaller.install()

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_argument('--headless')  # GUI 없이 실행 (headless 모드)
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('lang=ko_KR')

# ChromeDriver 실행
driver = webdriver.Chrome(options=chrome_options)

# 페이지 접속
url = "https://www.easylaw.go.kr/CSP/CsmSortRetrieveLst.laf?sortType=cate&csmAstSeq=12" # 근로
driver.get(url)
time.sleep(2)

# WebDriverWait
wait = WebDriverWait(driver, 10)

# li[] 리스트
li_list = [1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

titles = []
texts = []

for li in li_list:
    try:
        # 상세 페이지 클릭
        detail_page_xpath = f'//*[@id="contents"]/div[5]/ul/li[{li}]/a/span'
        detail_page_element = wait.until(EC.presence_of_element_located((By.XPATH, detail_page_xpath)))
        detail_page_element.click()
        time.sleep(2)  # 상세 페이지 로드 대기

        # 상세 페이지 제목 추출
        title_xpath = '//*[@id="pageTitle"]/h4[1]'
        title_element = wait.until(EC.presence_of_element_located((By.XPATH, title_xpath)))
        page_title = title_element.text

        # 상세 페이지 텍스트 추출
        content_xpath = '//*[@id="ovDiv"]/div[1]'
        content_element = wait.until(EC.presence_of_element_located((By.XPATH, content_xpath)))
        page_text = content_element.text

        titles.append(page_title)
        texts.append(page_text)

        driver.back()
        time.sleep(2)

    except Exception as e:
        print(f"{li}번째 항목에서 오류 발생: {e}")
        break  # 오류가 발생하면 루프 종료

# DataFrame에 저장
df = pd.DataFrame({
    "title": titles,
    "text": texts
})

# 크롤링된 데이터 출력
print(df)

# CSV 파일로 저장
df.to_csv('생활법령_근로.csv', index=False, encoding='utf-8-sig')

# WebDriver 종료
driver.quit()

                                     title  \
0                                  가사근로자란?   
1                              건설일용근로자의 의의   
2                                 고객응대업무란?   
3                           기간제 및 단시간근로자란?   
4                           플랫폼과 플랫폼 노무제공자   
5                              산업재해보상보험 개관   
6                   산업재해보상 보험급여의 개념 및 특징 등   
7                                시간선택제 일자리   
8                                 "실업급여"란?   
9                              여성의 고용차별 금지   
10                        외국인근로자 취업 및 체류자격   
11                          유연근무제의 의의 및 유형   
12                           근로시간 및 휴가권 보장   
13                           임금의 개념 및 판단기준   
14                                "장애인"이란?   
15                                 장애인의 개념   
16                 지원을 받을 수 있는 청년은 몇 살인가요?   
17                                     퇴직금   
18  파견근로자의 개념 및 「파견근로자 보호 등에 관한 법률」의 적용 범위   
19                            근로관계의 종료와 해고   

                                 